In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from math import sqrt, ceil
from timeit import default_timer as timer

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

%matplotlib inline

In [4]:
# Open data file
with open("/content/drive/MyDrive/CNN/data2.pickle", 'rb') as f:
    data = pickle.load(f, encoding="latin1")

# Prepare for keras
data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

# Making channels come at the end
data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

In [12]:
filters = [3, 5, 9]
model = [0] * len(filters)

for i in range(len(model)):
  model[i] = Sequential()
  model[i].add(Conv2D(32, kernel_size=filters[i], padding='same', activation='relu', input_shape=(32, 32, 3)))
  model[i].add(BatchNormalization())
  model[i].add(MaxPool2D(pool_size=2))
  model[i].add(Conv2D(256, kernel_size=filters[i], padding='same', activation='relu'))
  model[i].add(BatchNormalization())
  model[i].add(MaxPool2D(pool_size=2))
  model[i].add(Flatten())
  model[i].add(Dense(500, activation='relu'))
  model[i].add(Dropout(0.5))
  model[i].add(Dense(500, activation='relu'))
  model[i].add(Dropout(0.5))
  model[i].add(Dense(43, activation='softmax'))
  model[i].compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))
epochs = 5

h = [0] * len(model)
for i in range(len(h)):
  h[i]= model[i].fit(data['x_train'], data['y_train'],
                    batch_size=500, epochs = epochs,
                    validation_data = (data['x_validation'], data['y_validation']),
                    callbacks=[annealer], verbose=1)
  print('Model with filters {0:d}x{0:d}, epochs={1:d}, training accuracy={2:.5f}, validation accuracy={3:.5f}'.\
      format(filters[i], epochs, max(h[i].history['accuracy']), max(h[i].history['val_accuracy'])))

Epoch 1/5
174/174 [==============================] - 15s 82ms/step - loss: 3.1348 - accuracy: 0.2627 - val_loss: 7.6086 - val_accuracy: 0.0068
Epoch 2/5
174/174 [==============================] - 14s 79ms/step - loss: 0.6346 - accuracy: 0.8007 - val_loss: 7.6674 - val_accuracy: 0.0510
Epoch 3/5
174/174 [==============================] - 14s 79ms/step - loss: 0.2885 - accuracy: 0.9090 - val_loss: 3.4091 - val_accuracy: 0.3741
Epoch 4/5
174/174 [==============================] - 14s 79ms/step - loss: 0.1819 - accuracy: 0.9431 - val_loss: 0.1982 - val_accuracy: 0.9406
Epoch 5/5
174/174 [==============================] - 14s 79ms/step - loss: 0.1392 - accuracy: 0.9574 - val_loss: 0.1321 - val_accuracy: 0.9639
Model with filters 3x3, epochs=5, training accuracy=0.95973, validation accuracy=0.96395
Epoch 1/5
174/174 [==============================] - 19s 105ms/step - loss: 3.2125 - accuracy: 0.2585 - val_loss: 5.2714 - val_accuracy: 0.0438
Epoch 2/5
174/174 [==============================] -

In [14]:
import scipy.stats as sciStats

labels = []
for i in range(len(model)):
    temp = model[i].predict(data['x_test'])
    temp = np.argmax(temp, axis=1)
    labels.append(temp)
    
    # We compare predicted class with correct class for all input images
    # And calculating mean value among all values of following numpy array
    # By saying 'testing_accuracy == data['y_test']' we create numpy array with True and False values
    # 'np.mean' function will return average of the array elements
    # The average is taken over the flattened array by default
    temp = np.mean(temp == data['y_test'])*100
    
    print('data2 filter {0:d} testing accuracy = {1:.5f}%'.format(filters[i], temp))

# Ensemble with voting
labels = np.array(labels)
# labels = np.transpose(labels, (1, 0))
labels = sciStats.mode(labels)[0]
labels = np.squeeze(labels)
# np.set_printoptions(threshold=np.inf)

ensembleAccuracy = np.mean(labels == data['y_test'])*100

print('Ensemble Accuracy = {0:.5f}%'.format(ensembleAccuracy))

data2 filter 3 testing accuracy = 95.24941%
data2 filter 5 testing accuracy = 95.35234%
data2 filter 9 testing accuracy = 91.52811%
Ensemble Accuracy = 95.84323%
